# Monte Carlo Simulation in a Scattering Medium

In [ ]:
using Unitful
using JLD2
using CairoMakie
using Random

# Optional
#] add Distributions
using Distributions
using Base.Threads

### Sidenote: Multi-threaded Programming in Julia

Want to use multi-threads in Julia? You have to install a kernel with threads! Open Julia and do:

```{julia}
using IJulia
N = ???
installkernel("Julia N threads", env=Dict("JULIA_NUM_THREADS"=>N))
```

A rectangular box contains a medium with extinction $\alpha^a$ and $\alpha^s$ from absorption and scattering processes respectively. At the bottom of the box, photons are emitted vertically only from specific locations. The locations where photons are emitted are given by a mask saved in the file `RoCS_image.jld2`, which contains an integer array. Photons should be emitted only from locations where this array is one. When plotting, be sure to use `origin='lower'` so that the orientation is correct:

In [ ]:
@load "RoCS_image.jld2" data

heatmap(data', colormap=:gist_gray, axis=(aspect=DataAspect(),))

You can assume the physical size of each pixel is 1 m x 1 m. 

By means of a Monte Carlo simulation, we can study how the scattering and absorption in the medium affect the intensity measured at the top of the box (after crossing the medium). We simulate photons being emitted in pixels given by the mask above, initially all in the vertical direction. After a photon travels an optical depth of $\tau_0$, it will interact with matter and suffer one of two outcomes: be absorbed (destroyed) or be scattered. Scattered photons should be given a random new direction of travel, and then travel a new optical depth of $\tau_1$ until they have another interaction with matter. This simulation of the photon trajectory should end when the photon is absorbed or it exits the box. Photons that exit through the sides or bottom are ignored, and photons that cross the top boundary are recorded. We use these to build an image of the intensity at the top of the box. 
    
<img style="float: right;" src="https://tiagopereira.space/ast4310_h24/images/pattern.svg" width=300px>
    
The optical depth $\tau_i$ that a photon travels at each step $i$ should be probabilistically determined in the following way: $\tau_i = -\ln(\xi)$, where $\xi$ is a random number uniformly distributed in the interval (0, 1]. The type of interaction (absorption or scattering) will also be probabilistically determined, based on the photon destruction probability $\varepsilon$.

There are essentially three free parameters here: $\alpha^a$, $\alpha^s$, and the height of the box. Feel free to change them as you see fit. The main objective here is to study how the different parameters affect the intensity at the top of the box.

In [ ]:
################
################
# These functions are not completed. Fill in the missing code
################
################


"""
    monte_carlo(
        x::AbstractFloat,
        y::AbstractFloat,
        nx::Integer,
        ny::Integer,
        ε::AbstractFloat,
        α_tot::AbstractFloat,
        h::Real;
        z::Real=0
    )

Monte Carlo Simulation of photons going through a scattering medium. 
Returns position of photon if it escapes.
"""
function monte_carlo(
        x::Integer,
        y::Integer,
        nx::Integer,
        ny::Integer,
        ε::AbstractFloat,
        α_tot::AbstractFloat,
        h::Real;
        z::Real=0
    )

    hit = 1
    # Initial setup
    θ = 0     # photons emitted vertically
    φ = 0     # doesn't matter what phi is in the first iteration

    # Uniform distributions for the angles
    θ_dist = Uniform(0,π)
    φ_dist = Uniform(0,2π) 
    
    while z < h

        # photon destruction?
        ### TODO
        # Use Monte-Carlo to figure out if the photon is destroyed
        if ...
            hit = 0
            break

        # photon scatters
        else
            ### TODO
            # Find the length the photon will travel
            χ = ...
            τ = ...
            # l is the physical length the photon travels
            l = tau/alpha_tot
            
            x += l*sin(θ)*cos(φ)
            y += l*sin(θ)*sin(φ)
            z += l*cos(θ)
    
            # photon exits box
            if y < 0 || x < 0 || z < 0 || y > ny || x > nx
                hit = 0
                break
            
            # assign random direction to scattered photon
            else
                φ = rand(φ_dist) 
                θ = rand(θ_dist)

                # Alternative
                # φ = rand() * 2π
                # θ = rand() * π
            end
        end
    end
        
    return floor(Int,x), floor(Int,y), hit
end

"""
    calculate_scattering(
        image::Matrix,
        alpha_a::AbstractFloat,
        alpha_s::AbstractFloat,
        h::AbstractFloat,
        trials::Integer=50
    )

Calculates how the RoCS picture looks like through a medium.
Returns diffuse image.
"""
function calculate_scattering(
            img::Matrix,
            alpha_a::AbstractFloat,
            alpha_s::AbstractFloat,
            h::Real;
            n_trials::Integer=50
    )
    
    nx, ny = size(img)

    ### TODO
    # Calculate total extinction and destruction probability
    alpha_tot = ...
    epsilon = ...
    
    diffuse_image = zero(img)

    # Loop over pixels that emit photons only
    for ind in findall(img .!= 0)
        for k in 1:n_trials
            idx, idy, hit = monte_carlo(ind[1],ind[2],nx,ny,epsilon,alpha_tot,h)
            diffuse_image[idx, idy] += hit
        end
    end
            
    return diffuse_image
end

In [ ]:
# Set random seed for reproducibility
Random.seed!(4310)

# Trying with some different values for absorption coefficient while having fixed scattering


In [ ]:
# Plot the diffuse image
f = Figure(size=(1500,800))

ax1 = Axis(f[1,1], aspect=DataAspect(), title="Bottom of box")
heatmap!(ax1, data' / 255, colorrange=(0,1),colormap=Reverse(:binary), rasterize=true)

ax2 = Axis(f[1,2], aspect=DataAspect(), title=L"$\alpha^a = 0.1$")
heatmap!(ax2, rocs_pic1', colorrange=(0,1),colormap=Reverse(:binary), rasterize=true)

ax3 = Axis(f[2,1], aspect=DataAspect(), title=L"$\alpha^a = 0.01$")
heatmap!(ax3, rocs_pic2', colorrange=(0,1),colormap=Reverse(:binary), rasterize=true)

ax4 = Axis(f[2,2], aspect=DataAspect(), title=L"$\alpha^a = 0.001$")
heatmap!(ax4, rocs_pic3', colorrange=(0,1),colormap=Reverse(:binary), rasterize=true)

f

In [ ]:
using BenchmarkTools
# Test performance ...

Some points we focus on:
    
* For a given height of the box, what typical parameters are needed for most of the image information to reach the top? 
* When do the letters stop being visible, and when does the circle?
* How many mean free paths do the structures at the bottom need to be such that they are still visible at the top?
* How do the photon destruction probability and scattering extinction affect the diffusion of the top image?

How far will a photon travel? Photon mean free path $l = 1/\alpha$

In [ ]:
# Plot the diffuse image
f = Figure(size=(900,700))
ax1 = Axis(f[1,1], aspect=DataAspect(), title="Bottom of box")


ax2 = Axis(f[2,1], aspect=DataAspect(), title="Top of box")


f